# Bank Customer Churn -  Ensemble Learning 


In [ ]:
#![churn_bank.jpg](attachment:churn_bank.jpg)

# Importando Bibliotecas 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import warnings
import logging
import optuna
import xgboost as xgb
import torch
import torch.nn as nn
import shap
import lime.lime_tabular
import scipy.sparse
from sklearn.inspection import PartialDependenceDisplay
from xgboost import plot_importance
from tqdm import tqdm
from datetime import datetime, timedelta
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler #, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score,
    roc_curve, confusion_matrix, auc
)

from optuna.samplers import TPESampler
import pickle
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import joblib

from imblearn.over_sampling import BorderlineSMOTE
from tensorflow.keras.callbacks import EarlyStopping
import math






########################### para KAGGLE ################################################################################################################
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
########################################################################################################################################################


# Dicinário de Dados 


| **Variável**         | **Tipo**   | **Descrição**                                                                                                                                     |
|-----------------------|------------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| RowNumber            | int64      | Número do registro (linhas), sem efeito na construção de modelos.                                                                                |
| CustomerId           | int64      | ID do cliente, sem efeito sobre o estudo.                                                                                                       |
| Surname              | object     | Sobrenome do cliente, sem impacto na análise.                                                                                                   |
| CreditScore          | int64      | Pontuação de crédito, pode indicar tendência de permanência de clientes com pontuação alta.                                                     |
| Geography            | object     | Localização do cliente, pode influenciar a decisão de evasão.                                                                                   |
| Gender               | object     | Gênero do cliente, possível influência na evasão.                                                                                               |
| Age                  | int64      | Idade do cliente, clientes mais velhos tendem a permanecer.                                                                                     |
| Tenure               | int64      | Anos que o cliente está no banco, clientes novos têm maior chance de evasão.                                                                    |
| Balance              | float64    | Saldo na conta, pessoas com saldos altos são menos propensas a sair.                                                                            |
| NumOfProducts        | int64      | Número de produtos adquiridos pelo cliente.                                                                                                    |
| HasCrCard            | int64      | Indica se o cliente tem cartão de crédito, clientes com cartão são menos propensos à evasão.                                                    |
| IsActiveMember       | int64      | Clientes ativos têm menor chance de evasão.                                                                                                    |
| EstimatedSalary      | float64    | Salário estimado, clientes com salários mais altos tendem a permanecer.                                                                         |
| Exited               | int64      | Indica se o cliente saiu ou não do banco, variável de predição (“churn”).                                                                       |
| Complain             | int64      | Indica se o cliente fez reclamação.                                                                                                             |
| Satisfaction Score   | int64      | Pontuação de satisfação com a resolução de reclamação.                                                                                          |
| Card Type            | object     | Tipo de cartão que o cliente possui.                                                                                                            |
| Points Earned        | int64      | Pontos ganhos pelo cliente.                                                                                                                     |


# Análise Exploratória (EDA) & Data Wrangling 


In [ ]:

# base_original = pd.read_csv('/kaggle/input/Customer-Churn-Records.csv', sep=',') #KAGGLE
base_original = pd.read_csv('C:/Users/jgeov/iCloudDrive/Treinamento/Treinamento Data Science/Projetos/Customer-Churn-Records.csv',sep=',') #LOCAL

#configs para nao quebrar linhas no print do  df
pd.set_option('display.expand_frame_repr', False) 
pd.set_option('display.max_columns', None)

#primeiras linhas 
base_original.head()

Analisando primeiras impressões da base de dados


In [ ]:
#Dimensões 
print("Numero de linhas:", base_original.shape[0]) 
print("Numero de colunas:", base_original.shape[1])


In [ ]:
#tipos
base_original.dtypes

In [ ]:
#checando se há valores nulos 
base_original.isnull().sum()  
#valores nulos nao encontrados 

In [ ]:
#contando a quantidade de zeros em cada coluna para verificar se elas tem 
# informacao suficiente para entrar no modelo futuramente

for col in base_original.columns:
    zero_count = (base_original[col] == 0).sum()
    print("")
    print(f" '{col}': {zero_count} valores zero")

#nao foi constatao nada muito impactante, as variaveis com mais zero sao categoriacas (binarias) 
# Balance seria a unica a se preocupar, mas vamos manter. 





In [ ]:
#removidas por serem meramente identificadoras: RowNumber, CustomerId e Surname

#removida Gender por poder inviesar o modelo de alguma formna descriminativa, é uma boa pratica de LGPD nao usar dados sensiveis como esse etc. 
 

df = base_original[['CreditScore',
                    #'Gender',
                    'Geography',
                    'Age',
                    'Tenure',
                    'Balance',
                   'NumOfProducts',
                    'HasCrCard',
                    'IsActiveMember',
                   'EstimatedSalary',
                    'Complain',
                    'Satisfaction Score',
                   'Card Type',
                    'Point Earned',
                    'Exited'
                   ]]

# Resumo estatístico 
quanti = df[['EstimatedSalary', 'Balance', 'CreditScore', 'Age', 'Tenure', 'Point Earned']]
resumo_estati_quant = quanti.describe().style.format(lambda x: f'{x:,.1f}'.replace(',', 'X').replace('.', ',').replace('X', '.')) # Formatação com 1 casa decimal e separadores invertidos

resumo_estati_quant

* Resumo estatistico de variaveis qualitativas (frequancias)

* Os resumos estatisticos sao importantes para primeiras nocoes de desbalance, a amplitude e distribuicao de valores
minimos maximos e um breve entendimento se serao necessarios tratamentos nessas variaveis, decorrentes dessas observacoes; 

* Podemos notar que a principio as ditribuicoes nao sao absurdas e o desbalance esta pricipalemnte nas variavies Complain e Exited (variavel alvo do estudo, a chamaremos de churn) indicando que sera necessario tratar isso;

* Franca tem mais observacoes que os demais paises; 

* A maioria dos clientes tem cartao de credito; 

* A maioria dos clientes tem entre 1 e 2 produtos. 


In [ ]:
#Resumo estatistico 

#separando quali's para analise 
quali = df[['HasCrCard', 
            'IsActiveMember', 
            'Geography',
            #'Gender',
            'Complain',
            'Exited',
            'Card Type',
            'NumOfProducts',
            'Satisfaction Score']]

quali = quali.astype('object')

#quali.dtypes



def add_value_labels(ax):
    for p in ax.patches:
        height = p.get_height()
        color = p.get_facecolor()
        ax.text(p.get_x() + p.get_width() / 2., height / 2.,
                f'{int(height)}',
                ha='center', va='center', fontsize=20, color='white', fontweight='bold',
                bbox=dict(facecolor=color, edgecolor='none', alpha=0.7,
                          boxstyle='round,pad=0.4', linewidth=1))

plt.figure(figsize=(20, 25))

# Geography
plt.subplot(5, 2, 1)
ax1 = plt.gca()
ax1.set_title('Geography', fontsize=22, fontweight='bold')
sns.countplot(x='Geography', hue='Geography', palette='viridis', data=base_original, ax=ax1, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax1)


# Complain
plt.subplot(5, 2, 2)
ax10 = plt.gca()
ax10.set_title('Complain', fontsize=22, fontweight='bold')
sns.countplot(x='Complain', hue='Complain', palette='viridis', data=base_original, ax=ax10, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax10)

# HasCrCard
plt.subplot(5, 2, 3)
ax5 = plt.gca()
ax5.set_title('HasCrCard', fontsize=22, fontweight='bold')
sns.countplot(x='HasCrCard', hue='HasCrCard', palette='viridis', data=base_original, ax=ax5, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax5)

# IsActiveMember
plt.subplot(5, 2, 4)
ax6 = plt.gca()
ax6.set_title('IsActiveMember', fontsize=22, fontweight='bold')
sns.countplot(x='IsActiveMember', hue='IsActiveMember', palette='viridis', data=base_original, ax=ax6, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax6)

# Card Type
plt.subplot(5, 2, 5)
ax10 = plt.gca()
ax10.set_title('Card Type', fontsize=22, fontweight='bold')
sns.countplot(x='Card Type', hue='Card Type', palette='viridis', data=base_original, ax=ax10, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax10)


# NumOfProducts
plt.subplot(5, 2, 6)
ax10 = plt.gca()
ax10.set_title('NumOfProducts', fontsize=22, fontweight='bold')
sns.countplot(x='NumOfProducts', hue='NumOfProducts', palette='viridis', data=base_original, ax=ax10, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax10)


# Satisfaction Score
plt.subplot(5, 2, 7)
ax11 = plt.gca()
ax11.set_title('Satisfaction Score', fontsize=22, fontweight='bold')
sns.countplot(x='Satisfaction Score', hue='Satisfaction Score', palette='viridis', data=base_original, ax=ax11, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax11)






# Exited
plt.subplot(5, 2, 8)
ax11 = plt.gca()
ax11.set_title('Exited: Churn ', fontsize=22, fontweight='bold')
custom_palette = ['green', 'red']
sns.countplot(x='Exited', hue='Exited', palette=custom_palette, data=base_original, ax=ax11, legend=False)
plt.xlabel('') 
plt.ylabel('') 
plt.xticks(fontsize=15, rotation=0, fontweight='bold')
plt.yticks([])
add_value_labels(ax11)

ax11.set_xticks([0, 1])
ax11.set_xticklabels(['Não', 'Sim'], fontsize=15, fontweight='bold')

# Ajustando espaçamento
plt.subplots_adjust(hspace=0.3, wspace=0.1)

plt.show()




* Visualizando o comportmento da variavel alvo (exited) em relacao as demais variaveis; 

* Vemos claramente que existe o disbalance de classes na variavel churn, pela cor verde presente fortemente em todas variaveis, posteriormente isso sera tratado/mitigado; 

* Ja e possivel notar um forte indicio de alta correlacao entre churn e complain, posteriormente isso sera testado. 



In [ ]:
#%% Variável alvo em relação as demais variáveis 





plt.figure(figsize=(20, 25)) #tamanho do painel grafico

#funcao de adicao de legenda no canto superior direito e garante rotulos 
def add_legend(ax):
    
    handles, labels = ax.get_legend_handles_labels()
    if not handles:
        
        # Se não houver handles, forca a adicao
        handles = [plt.Rectangle((0,0),1,1, color=c) for c in ['green', 'red']]
        labels = ['Not Exited', 'Exited']
        
    # Adiciona a legenda fora da área das barras
    ax.legend(handles, labels, loc='upper left', fontsize=14, title='Exited', title_fontsize='13',  
              bbox_to_anchor=(1.0, 1)) 




# Geography
plt.subplot(5, 2, 1)
counts = base_original.groupby(['Geography', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Geography', fontsize=22, fontweight='bold')
plt.xlabel('Geography', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)


# NumOfProducts
plt.subplot(5, 2, 2)
counts = base_original.groupby(['NumOfProducts', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by NumOfProducts', fontsize=22, fontweight='bold')
plt.xlabel('NumOfProducts', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# HasCrCard
plt.subplot(5, 2, 3)
counts = base_original.groupby(['HasCrCard', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by HasCrCard', fontsize=22, fontweight='bold')
plt.xlabel('HasCrCard', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# IsActiveMember
plt.subplot(5, 2, 4)
counts = base_original.groupby(['IsActiveMember', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca()) 
plt.title('Exited by IsActiveMember', fontsize=22, fontweight='bold')
plt.xlabel('IsActiveMember', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Complain
plt.subplot(5, 2, 5)
counts = base_original.groupby(['Complain', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Complain', fontsize=22, fontweight='bold')
plt.xlabel('Complain', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Satisfaction Score
plt.subplot(5, 2, 6)
counts = base_original.groupby(['Satisfaction Score', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Satisfaction Score', fontsize=22, fontweight='bold')
plt.xlabel('Satisfaction Score', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Card Type
plt.subplot(5, 2, 7)
counts = base_original.groupby(['Card Type', 'Exited']).size().unstack().fillna(0)
ax = counts.plot(kind='bar', stacked=True, color=['green', 'red'], ax=plt.gca())  
plt.title('Exited by Card Type', fontsize=22, fontweight='bold')
plt.xlabel('Card Type', fontsize=16)
plt.ylabel('', fontsize=16)
plt.xticks(fontsize=14, rotation=0,fontweight='bold')
plt.yticks(fontsize=14)
add_legend(ax)

# Ajusta a distância entre os gráficos
plt.subplots_adjust(hspace=0.7, wspace=0.3)

# Variaveis Dummies

* A maioria dos modelos necessita de transformar as variaveis categoricas em numericas, e o modelo atual e um deles; 

* A transformacao de categoricas em numericas precisa ser feita com processos adequados para nao cometer ponderacao arbitrária no desenvolvimento. 

* foi aplicado one-hot encoding para isso. Esse processo e chamado de "Dummizacao". 

* foi necessario aplicar Ordinal-Encoder. 

In [ ]:
# Dumizando

# Suprime todos os warnings de futuro (deixa mais clean)
warnings.filterwarnings('ignore', category=FutureWarning)

# Lista de variáveis a serem transformadas
cols_to_transform = ['Geography']

# Convertendo para string (somente a coluna "Geography")
df.loc[:, cols_to_transform] = df.loc[:, cols_to_transform].astype(str)

# Realizando o One-Hot Encoding 
df_dummies = pd.get_dummies(df, columns=cols_to_transform, dtype=int, drop_first=True) 
#se o modelo/modelos forem afetados drasticamntee por multicolineariadde drop_first=True deve ser melhor, pois dropa uma das variaveis dummie. Os demais nao feta tanto, mas e boa pratica

# Ordinal-Encoder 
df_dummies['Card Type'] = OrdinalEncoder(categories=[["SILVER", "GOLD", "PLATINUM", "DIAMOND"]], dtype=int).fit_transform(df_dummies[['Card Type']])

# Variável alvo 'Exited' para o tipo numérico (se necessário)
df_dummies['Exited'] = df_dummies['Exited'].astype('int64')

print(df_dummies.dtypes)

# Separação Treino e Teste & Adicao de Features quadráticas

* A separacao em treino e teste alem de uma boa pratica e extreamente necessario na construcao de modelos de machine learning; 

* Tambem foram adicionadas variaveis quadraticas, ou seja, com operacao matematica aplicadas em variaveis originais gerando novas variaveis. Isso foi feito para capturar algum tipo de comportamento nao linear; 

* Foi considerado aplicar transformacao polinomial nas variaveis, por isso foi primeiro aplicado o termo quadratico, que nao apresentou melhoria significativa a ponto de aplicarmos polinomias; 

* Alem disso esse modelo captura naturalmente comportamentos nao lineares. O termo quadratico foi util para validacao durante o estudo mas a melhoria foi baixa, por isso mantemos apenas os termos quadraticos sem incluir interacoes entre variaveis (seria aplicacao Polinomias completo); 



* Tambem e possivel notar o desbalanceamento das classes  nas bases tanto em treino quanto em teste no grafico final; 

* Tambem garantimos a mesma proporcao (80/20) tanto em treino quanto em teste na separacao das bases , ou seja, equidade de divisao de dados e equilibrio. 


In [ ]:
#X ---> Variáveis explicativas 
#Y ---> Evento de estudo (variável TARGET, evento de estudo, ^y etc..)

df_dummies = df_dummies.drop(columns=['Complain']) #correlacao altissima com a variavel alvo


X = df_dummies .drop('Exited', axis=1)

# VARIAVEIS QUADRATICAS 
X['Balance_Squared'] = X['Balance'] ** 2
X['Age_Squared'] = X['Age'] ** 2
X['CreditScore_Squared'] = X['CreditScore'] ** 2
X['Tenure_Squared'] = X['Tenure'] ** 2
X['EstimatedSalary_Squared'] = X['EstimatedSalary'] ** 2



y =  df_dummies['Exited']


#separando em treino e teste 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Visualizando a proporção de eventos de churn (TARGET) nas bases de TREINO e TESTE 

# Contando os valores 
churn_counts_train = y_train.value_counts()
churn_counts_test = y_test.value_counts()


# plot que contem os graficos
fig, axs = plt.subplots(1, 2, figsize=(20, 10))
fig.suptitle('Proporção da Variável Churn entre Treino e Teste', fontsize=35)  
cmap = plt.get_cmap('viridis', 2) #paleta de cores



# Gráfico da base de treino
bars_train = axs[0].bar(churn_counts_train.index, churn_counts_train.values, color=cmap(range(2)))
axs[0].set_title('Base de Treino', fontsize=25)
axs[0].set_xlabel('Churn', fontsize=20)
axs[0].set_ylabel('Contagem', fontsize=20)
axs[0].set_xticks([0, 1])
axs[0].set_xticklabels(['0', '1'], fontsize=20)
axs[0].set_yticklabels([]) # Ocultando os valores do eixo y

# Adicionando rótulos de dados
total_train = churn_counts_train.sum()
for bar in bars_train:
    count = int(bar.get_height())
    percentage = round(count / total_train * 100)  # Arredonda a porcentagem
    label = f'{count} ({percentage}%)'  #valor absoluto e o percentual
    axs[0].text(bar.get_x() + bar.get_width() / 2, bar.get_height() / 2, 
                label, ha='center', color='gray', fontsize=25, weight='bold')




# Gráfico da base de teste
bars_test = axs[1].bar(churn_counts_test.index, churn_counts_test.values, color=cmap(range(2)))
axs[1].set_title('Base de Teste', fontsize=25)
axs[1].set_xlabel('Churn', fontsize=20)
axs[1].set_ylabel('Contagem', fontsize=20)
axs[1].set_xticks([0, 1])
axs[1].set_xticklabels(['0', '1'], fontsize=20)
axs[1].set_yticklabels([])# Ocultando os valores do eixo y

# Adicionando rótulos de dados
total_test = churn_counts_test.sum()
for bar in bars_test:
    count = int(bar.get_height())
    percentage = round(count / total_test * 100)  # Arredonda a porcentagem
    label = f'{count} ({percentage}%)'  #valor absoluto e o percentual
    axs[1].text(bar.get_x() + bar.get_width() / 2, bar.get_height() / 2, 
                label, ha='center', color='gray', fontsize=25, weight='bold')



# Ajusta o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0, 1, 0.95])  # forca espaco para o titulo
plt.show()




# Verificando correlacoes depois de construcao total de features e Dummizacao 

* Verificar as correlacoes e extreammente importante, elas podem indicar a famosa multicolinearidade, que atrapalha a maioria dos modelos; 

* no caso da deste modelo (pelo menos a presente aplicacao) ela nao afetou, a observei com atencao, mas nao impactou. Decidi manter as variveis mesmo com multicolinearidade em algumas (nao se assuste). 



In [ ]:
#Observando Multicolinearidade na base de treino

teste_multco_treino = pd.concat([X_train,y_train], axis = 1)

correlation_matrix_treino = teste_multco_treino.corr().round(2)
correlation_matrix_treino

# Matrix com  mapa de calor 
plt.figure(figsize=(30, 20))
heatmap = sns.heatmap(correlation_matrix_treino, annot=True, fmt=".2f",
                      cmap=plt.cm.viridis_r, # paleta de cores viridis (ou viridis_r para o inverso de cores) é uma paleta especial 
                                             # para facilitar a visualizacao por pessoas com dificuldades visuais, como os daltonicos. 
                      annot_kws={'size': 15}, vmin=-1, vmax=1)
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=17)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=17)
plt.title('Correlação das Variáveis Quantitativas na Base de Treino',fontsize=25)
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=17)
plt.show()

# Analise e tratamento de Outliers 

* Outliers sao numericos e podem afetar de diversas formas modelos; 

* para resolver sem perder dados, pois temos poucas observcoes para estudo, nao foram removidos como facilmente poderia fazer-se, ao inves foi aplicado winsorization; 

* winsorization e uma tecnica de limitacao dos outliers, ela substiui os valores de outlierns pelos limites superiores e inferiores; 

* Para isso, e calculado um intervalo de valores aceitos com base no primeiro quartil (Q1) e no terceiro quartil (Q3), valores abaixo do limite inferior ou acima do limite superior sao ajustados para os respectivos limites, corrigindo assim os outliers 

In [ ]:
#%% analise de outliers das variaveis na base de treino 


###############antes de tratamento############################# 
variaveis = [
    'CreditScore',
    'Age',
    'Tenure',
    'Balance',
    'NumOfProducts',
    'EstimatedSalary',
    'Satisfaction Score',
    'Point Earned',
    #QUADRATICAS
    'Balance_Squared',
    'Age_Squared',
    'CreditScore_Squared',
    'Tenure_Squared',
    'EstimatedSalary_Squared'
]


# definindo tamnhos de subplots 
plt.figure(figsize=(16, 12))

# loop de criacao de boxplots para cada variavel 
for i, var in enumerate(variaveis):
    plt.subplot(5, 4, i + 1)  #determina a grade de plots 
    sns.boxplot(y=teste_multco_treino[var],
               boxprops=dict(facecolor='lightblue'))  # Cor interna do boxplot 
    plt.title(f'Boxplot {var}', fontsize=12)
    
#  título geral
plt.suptitle('Análise de Outliers nas Variáveis(treino) - antes de "winsorization" ', fontsize=20)

plt.tight_layout(rect=[0, 0, 1, 0.95]) # Ajuste de layout
plt.show()
###############################################################






# Função que aplica winsorization
def tratar_outliers(df, coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    # Substitui outliers pelo limite inferior ou superior
    df[coluna] = np.where(df[coluna] < limite_inferior, limite_inferior, df[coluna])
    df[coluna] = np.where(df[coluna] > limite_superior, limite_superior, df[coluna])





###############depois de tratamento############################# 

# Aplicando a função nas variáveis 
variaveis_para_tratar = ['CreditScore',
                        'Age',
                        'Tenure',
                        'Balance',
                        'NumOfProducts',
                        'EstimatedSalary',
                        'Satisfaction Score',
                        'Point Earned',
                        #QUADRATICAS
                        'Balance_Squared',
                        'Age_Squared',
                        'CreditScore_Squared',
                        'Tenure_Squared',
                        'EstimatedSalary_Squared'
                         ]
for variavel in variaveis_para_tratar:
    tratar_outliers(teste_multco_treino, variavel)


# subplot
plt.figure(figsize=(16, 12))



# loop de criacao de boxplots para cada variavel 
for i, var in enumerate(variaveis):
    plt.subplot(5, 4, i + 1)  #determina a grade de plots 
    sns.boxplot(y=teste_multco_treino[var],
               boxprops=dict(facecolor='green'))  # Cor interna do boxplot
    plt.title(f'Boxplot {var}', fontsize=12)
########################################################################


# título geral
plt.suptitle('Análise de Outliers nas Variáveis(treino) - depois de "winsorization" ', fontsize=20)

plt.tight_layout(rect=[0, 0, 1, 0.95])  # ajusta layout
plt.show()

In [ ]:
#BASE DE TREINO 
teste_multco_treino

In [ ]:
# BASE DE TESTE
base_corrige_teste = pd.concat([X_test,y_test], axis = 1)
base_corrige_teste


# Preparacao dos dados  


* Preparacao dos dados 


In [ ]:
print('------------------------')
print(" DATA Prep")
print('------------------------')

# Desativando os warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module='torch')
warnings.filterwarnings("ignore", category=UserWarning, module='optuna')
warnings.filterwarnings("ignore", category=FutureWarning, module='optuna')
warnings.filterwarnings('ignore', category=DeprecationWarning)
logging.getLogger("optuna").setLevel(logging.CRITICAL)



# início
start_time_utc = datetime.utcnow() - timedelta(hours=3)
print('------------------------')
print("Início:", start_time_utc)
print('------------------------')


######################################## PRE-PROCESSAMENTO E PREPARACAO NOS DADOS ################################################################### 



# Defini variáveis de treinamento
X_train = teste_multco_treino.drop('Exited', axis=1)
y_train = teste_multco_treino['Exited']

# Verifica e conserta desalinhamento de índices caso tenha (X_test e y_test)
if not X_test.index.equals(y_test.index):
    print("Índices de X_test e y_test não estavam alinhados. Realinhando y_test.")
    y_test = y_test.loc[X_test.index]
else:
    print("Índices de X_test e y_test já estavam alinhados.")

# Concatena os dados corrigidos para criar a base de teste
base_corrige_teste = pd.concat([X_test, y_test], axis=1)

# Redefini X_test e y_test com índices corrigidos e verificados
X_test = base_corrige_teste.drop('Exited', axis=1)
y_test = base_corrige_teste['Exited']




# Criar e treinar o modelo Random Forest para selecionar as melhores variaveis 
#detalhe importante, nao tem predict(), somente o fit() 
rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

# Obter a importância das features
feature_importances = rf.feature_importances_

# Criar DataFrame com os nomes das features e suas importâncias
feature_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Filtrar features com importância maior que 0.01 (ou outro threshold desejado)
selected_features = feature_df[feature_df['Importance'] > 0.01]['Feature'].tolist()

# Ordenar as features pela importância (do menor para o maior)
feature_df = feature_df.sort_values(by='Importance', ascending=True)

# Criar o gráfico de barras horizontais
plt.figure(figsize=(10, 6))
plt.barh(feature_df['Feature'], feature_df['Importance'], color='skyblue')
plt.xlabel('Importância')
plt.ylabel('Features')
plt.title('Importância das Features Selecionadas (Random Forest)')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.xticks(np.arange(0, max(feature_df['Importance'])+0.01, 0.01))


# Mostrar o gráfico
plt.show()




# Finalizando preparacao e selecionando variaveis

Variaveis selecionadas conforme o modelo classificador de importancia usado anteriormente (randomforest) 

In [ ]:

print("Shape X_train antes de selecionar as fetuares importantes:", X_train.shape)
print("Shape y_train antes de selecionar as fetuares importantes:", y_train.shape)


warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.CRITICAL)


start_time_utc = datetime.utcnow() - timedelta(hours=3)
print("Tempo de Início:", start_time_utc)
print('------------------------')


X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

X_train, y_train = X_train_selected , y_train
X_test, y_test = X_test_selected , y_test

print("Shape X_train (selecionadas):", X_train.shape)
print("Shape y_train (selecionadas):", y_train.shape)

print("Shape X_test (selecionadas):", X_test.shape)
print("Shape y_test (selecionadas):", y_test.shape)


#verificando alinhamneto
assert X_train.shape[0] == y_train.shape[0], "Erro: Número de amostras não coincide"
assert X_test.shape[0] == y_test.shape[0], "Erro: Número de amostras não coincide"


# **Ensemble Model - Aplicando modelos salvos anteriormente e conferindo resultados** 

In [ ]:
#Catboost

import pickle
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score, confusion_matrix, roc_curve
)


# Caminhos dos arquivos
modelo_path = r"C:\Users\jgeov\OneDrive\Documentos\GitHub\Ciencia_de_dados-1\Churn_predict\CATboost\catboost_model.pkl"
scaler_path = r"C:\Users\jgeov\OneDrive\Documentos\GitHub\Ciencia_de_dados-1\Churn_predict\CATboost\scaler.pkl"

# Carregar o modelo treinado
with open(modelo_path, 'rb') as file:
    model_catboost = pickle.load(file)

# Carregar o scaler
with open(scaler_path, 'rb') as file:
    scaler_catboost = pickle.load(file)

# Normalizar os dados de teste
X_test_scaled = scaler_catboost.transform(X_test_selected)

# Fazer previsões
y_pred = model_catboost.predict(X_test_scaled)
y_pred_proba = model_catboost.predict_proba(X_test_scaled)[:, 1]  # Probabilidade da classe positiva

# Converter para DataFrame para análise
#df_pred = pd.DataFrame({
#    "y_real": y_test.values,  # Se `y_test` for DataFrame, pegar `.values`
#    "y_pred": y_pred,
#    "y_pred_proba": y_pred_proba
#})

# Exibir as primeiras linhas das previsões
#print(df_pred.head())


# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)
mcc = matthews_corrcoef(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)

# Exibir as métricas no formato desejado
print("\n📊 MÉTRICAS DO MODELO CATboost nos dados de teste")
print("=" * 60)
print(f"{'Métrica':<20}{'Valor final':<15}")
print("-" * 60)
print(f"Acurácia{' ' * 11}{accuracy:.4f}")
print(f"Precisão{' ' * 12}{precision:.4f}")
print(f"Recall{' ' * 14}{recall:.4f}")
print(f"F1-Score{' ' * 12}{f1:.4f}")
print(f"AUC-ROC{' ' * 13}{auc_roc:.4f}")
print(f"MCC{' ' * 17}{mcc:.4f}")
print(f"Kappa de Cohen{' ' * 7}{kappa:.4f}")
print(f"Acurácia Balanceada {balanced_acc:.4f}")
print("=" * 60)

# Criando a figura e os subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot da Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negativo", "Positivo"], 
            yticklabels=["Negativo", "Positivo"], ax=axes[0])
axes[0].set_xlabel("Predito")
axes[0].set_ylabel("Real")
axes[0].set_title("Matriz de Confusão (TESTE) - CATboost")

# Plot da Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
auc_roc_value = auc(fpr, tpr)
axes[1].plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_roc_value:.4f})", color="blue")
axes[1].plot([0, 1], [0, 1], linestyle="--", color="gray")  # Linha diagonal
axes[1].set_xlabel("Taxa de Falsos Positivos (FPR)")
axes[1].set_ylabel("Taxa de Verdadeiros Positivos (TPR)")
axes[1].set_title("Curva ROC (TESTE)- CATboost")
axes[1].legend()

# Distribuição de probabilidades

# classe positiva
probs_pos = y_pred_proba  
# classe negativa
probs_neg = 1 - y_pred_proba

# Plot classe positiva 
sns.kdeplot(probs_pos, color='blue', ax=axes[2], label='Classe Positiva', fill=True, alpha=0.6)

# Plot classe negativa 
sns.kdeplot(probs_neg, color='red', ax=axes[2], label='Classe Negativa', fill=True, alpha=0.02, linewidth=0.30)

# Ajustando o gráfico
axes[2].set_title("Distribuição das Probabilidades para as Classes Positiva e Negativa (TESTE) - CATboost")
axes[2].set_xlabel("Probabilidade")
axes[2].set_ylabel("Densidade")
axes[2].legend()
# setando eixo de probabilidades entre 0 e 1
axes[2].set_xlim(0, 1)


# Ajuste de layout
plt.tight_layout()
plt.show()



In [ ]:
# MLP - PyTorch

# Classe MLP do modelo (reconstrução exata)
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes, activation, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        for units in hidden_layer_sizes:
            self.layers.append(nn.Linear(input_size, units))
            self.layers.append(nn.Dropout(p=dropout_rate))
            input_size = units
        self.output = nn.Linear(input_size, 1)
        self.activation_fn = self.get_activation_function(activation)

    def forward(self, x):
        for layer in self.layers:
            x = self.activation_fn(layer(x))
        x = self.output(x)
        return x

    def get_activation_function(self, activation):
        activation_dict = {
            'relu': torch.relu,
            'tanh': torch.tanh,
            'sigmoid': torch.sigmoid,
            'selu': torch.selu,
            'gelu': torch.nn.functional.gelu,
            'leaky_relu': torch.nn.functional.leaky_relu,
            'swish': torch.nn.functional.silu,
            'elu': torch.nn.functional.elu
        }
        return activation_dict.get(activation, torch.relu)

# Definir o dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carregar o scaler
scaler_mlp_torch = torch.load(r"C:\Users\jgeov\OneDrive\Documentos\GitHub\Ciencia_de_dados-1\Churn_predict\MLP-Pytorch\scaler.pth")

#salvando para uso dentro desse notebook, uma copia com formato de joblib / pickle
joblib.dump(scaler_mlp_torch, "scaler_mlp_temp.pkl")

#definindo scaler definitivamente
scaler_mlp_torch = joblib.load("scaler_mlp_temp.pkl")


#apagando scaler_mlp_temp para nao confundir com scaler scaler oficial do ensemble

arquivo = "scaler_mlp_temp.pkl"
if os.path.exists(arquivo):
    os.remove(arquivo)
    print(f"Arquivo {arquivo} removido com sucesso.")
else:
    print(f"Arquivo {arquivo} não encontrado.")



# Normalizar os dados de teste
X_test_scaled = scaler_mlp_torch.transform(X_test)

# Converter X_test para tensor do PyTorch
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# Carregar os hiperparâmetros do modelo (ajuste isso se souber os valores exatos)
hidden_layer_sizes = [950, 850]  # com base nos melhores parâmetros encontrados no modelo carregado e treinado anteriormente 
activation = "relu"  # com base nos melhores parâmetros encontrados no modelo carregado e treinado anteriormente 
dropout_rate = 0.35657230019086544  # com base nos melhores parâmetros encontrados no modelo carregado e treinado anteriormente 

# Criar a instância do modelo com os hiperparâmetros corretos
input_size = X_test.shape[1]
model_mlp_torch = MLP(input_size, hidden_layer_sizes, activation, dropout_rate)

# Carregar o modelo salvo
model_mlp_torch = torch.load(r"C:\Users\jgeov\OneDrive\Documentos\GitHub\Ciencia_de_dados-1\Churn_predict\MLP-Pytorch\best_model_inteiro.pth", map_location=device)
model_mlp_torch.to(device)
model_mlp_torch.eval()

# Fazer previsões
with torch.no_grad():
    # Aplicar a função sigmoide para garantir que as probabilidades estejam no intervalo [0, 1]
    y_pred_proba = torch.sigmoid(model_mlp_torch(X_test_tensor)).cpu().numpy().flatten()

# Converter probabilidades para rótulos binários (0 ou 1)
y_pred = (y_pred_proba >= 0.5).astype(int)


# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_proba)
mcc = matthews_corrcoef(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)

# Exibir as métricas no formato desejado
print("\n📊 MÉTRICAS DO MODELO MLP nos dados de teste")
print("=" * 60)
print(f"{'Métrica':<20}{'Valor final':<15}")
print("-" * 60)
print(f"Acurácia{' ' * 11}{accuracy:.4f}")
print(f"Precisão{' ' * 12}{precision:.4f}")
print(f"Recall{' ' * 14}{recall:.4f}")
print(f"F1-Score{' ' * 12}{f1:.4f}")
print(f"AUC-ROC{' ' * 13}{auc_roc:.4f}")
print(f"MCC{' ' * 17}{mcc:.4f}")
print(f"Kappa de Cohen{' ' * 7}{kappa:.4f}")
print(f"Acurácia Balanceada {balanced_acc:.4f}")
print("=" * 60)

# Criando a figura e os subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot da Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negativo", "Positivo"], 
            yticklabels=["Negativo", "Positivo"], ax=axes[0])
axes[0].set_xlabel("Predito")
axes[0].set_ylabel("Real")
axes[0].set_title("Matriz de Confusão (TESTE) - MLP")

# Plot da Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
auc_roc_value = auc(fpr, tpr)
axes[1].plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_roc_value:.4f})", color="blue")
axes[1].plot([0, 1], [0, 1], linestyle="--", color="gray")  # Linha diagonal
axes[1].set_xlabel("Taxa de Falsos Positivos (FPR)")
axes[1].set_ylabel("Taxa de Verdadeiros Positivos (TPR)")
axes[1].set_title("Curva ROC (TESTE) - MLP")
axes[1].legend()

# Distribuição de probabilidades

# classe positiva
probs_pos = y_pred_proba  
# classe negativa
probs_neg = 1 - y_pred_proba

# Plot classe positiva 
sns.kdeplot(probs_pos, color='blue', ax=axes[2], label='Classe Positiva', fill=True, alpha=0.6)

# Plot classe negativa 
sns.kdeplot(probs_neg, color='red', ax=axes[2], label='Classe Negativa', fill=True, alpha=0.02, linewidth=0.30)

# Ajustando o gráfico
axes[2].set_title("Distribuição das Probabilidades para as Classes Positiva e Negativa (TESTE) - MLP")
axes[2].set_xlabel("Probabilidade")
axes[2].set_ylabel("Densidade")
axes[2].legend()
# setando eixo de probabilidades entre 0 e 1
axes[2].set_xlim(0, 1)

# Ajuste de layout
plt.tight_layout()
plt.show()



In [ ]:
# XGBoost 
import joblib
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc, matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Caminhos dos arquivos
model_path = r"C:\Users\jgeov\OneDrive\Documentos\GitHub\Ciencia_de_dados-1\Churn_predict\Xgboost\xgb_model.pkl"
scaler_path = r"C:\Users\jgeov\OneDrive\Documentos\GitHub\Ciencia_de_dados-1\Churn_predict\Xgboost\scaler.pkl"

# Carregar o scaler
scaler_xgb = joblib.load(scaler_path)

# Normalizar os dados de teste
X_test_scaled = scaler_xgb.transform(X_test)

# Carregar o modelo XGBoost
xgb_model = joblib.load(model_path)

# Fazer previsões (probabilidades)
y_pred_prob = xgb_model.predict(xgb.DMatrix(X_test_scaled))

# Converter para classe binária (usando threshold 0.5)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
mcc = matthews_corrcoef(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)

# Exibir métricas no formato esperado
print("\n📊 MÉTRICAS DO MODELO XGBOOST nos dados de teste")
print("="*60)
print(f"{'Métrica':<22} {'Valor final':<15}")
print("-"*60)
print(f"{'Acurácia':<22} {accuracy:.4f}")
print(f"{'Precisão':<22} {precision:.4f}")
print(f"{'Recall':<22} {recall:.4f}")
print(f"{'F1-Score':<22} {f1:.4f}")
print(f"{'AUC-ROC':<22} {roc_auc:.4f}")
print(f"{'MCC':<22} {mcc:.4f}")
print(f"{'Kappa de Cohen':<22} {kappa:.4f}")
print(f"{'Acurácia Balanceada':<22} {balanced_acc:.4f}")
print("="*60)

# Criando a figura e os subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot da Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negativo", "Positivo"], 
            yticklabels=["Negativo", "Positivo"], ax=axes[0])
axes[0].set_xlabel("Predito")
axes[0].set_ylabel("Real")
axes[0].set_title("Matriz de Confusão (TESTE) - XGBoost")

# Plot da Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc_value = auc(fpr, tpr)
axes[1].plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc_value:.4f})", color="blue")
axes[1].plot([0, 1], [0, 1], linestyle="--", color="gray")  # Linha diagonal
axes[1].set_xlabel("Taxa de Falsos Positivos (FPR)")
axes[1].set_ylabel("Taxa de Verdadeiros Positivos (TPR)")
axes[1].set_title("Curva ROC (TESTE) - XGBoost")
axes[1].legend()

# Distribuição de probabilidades

# classe positiva
probs_pos = y_pred_prob  
# classe negativa
probs_neg = 1 - y_pred_prob

# Plot classe positiva 
sns.kdeplot(probs_pos, color='blue', ax=axes[2], label='Classe Positiva', fill=True, alpha=0.6)

# Plot classe negativa 
sns.kdeplot(probs_neg, color='red', ax=axes[2], label='Classe Negativa', fill=True, alpha=0.02, linewidth=0.30)

# Ajustando o gráfico
axes[2].set_title("Distribuição das Probabilidades para as Classes Positiva e Negativa (TESTE) - XGBoost")
axes[2].set_xlabel("Probabilidade")
axes[2].set_ylabel("Densidade")
axes[2].legend()
# setando eixo de probabilidades entre 0 e 1
axes[2].set_xlim(0, 1)


# Ajuste de layout
plt.tight_layout()
plt.show()

In [ ]:
import torch
import xgboost as xgb
import catboost
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score, confusion_matrix, roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns


# ----------------------------------------------------------
# Normalizar os dados de teste e treino com os scalers corretos
# ----------------------------------------------------------

# XGBoost - Escalonamento com o scaler específico para XGBoost
X_train_scaled_xgb = scaler_xgb.transform(X_train)
X_test_scaled_xgb = scaler_xgb.transform(X_test)

# MLP (Torch) - Escalonamento com o scaler específico para MLP
X_train_scaled_mlp = scaler_mlp_torch.transform(X_train)
X_test_scaled_mlp = scaler_mlp_torch.transform(X_test)

# CatBoost - Escalonamento com o scaler específico para CatBoost
X_train_scaled_catboost = scaler_catboost.transform(X_train)
X_test_scaled_catboost = scaler_catboost.transform(X_test)

# ----------------------------------------------------------
# 1. XGBoost
# ----------------------------------------------------------

# Conjunto de treino
# Previsões no conjunto de treino
y_pred_prob_xgb_ensemble_train = xgb_model.predict(xgb.DMatrix(X_train_scaled_xgb))

# Conjunto de teste
# Fazer previsões (probabilidades)
y_pred_prob_xgb_ensemble_test = xgb_model.predict(xgb.DMatrix(X_test_scaled_xgb))

# Converter para classe binária (usando threshold 0.5)
xgb_y_pred_xgb_ensemble_test_ = (y_pred_prob_xgb_ensemble_test >= 0.5).astype(int)

# Calcular as métricas
xgb_ensemble_accuracy = accuracy_score(y_test, xgb_y_pred_xgb_ensemble_test_)
xgb_ensemble_precision = precision_score(y_test, xgb_y_pred_xgb_ensemble_test_)
xgb_ensemble_recall = recall_score(y_test, xgb_y_pred_xgb_ensemble_test_)
xgb_ensemble_f1 = f1_score(y_test, xgb_y_pred_xgb_ensemble_test_)
xgb_ensemble_auc_roc = roc_auc_score(y_test, y_pred_prob_xgb_ensemble_test)  # Correção aqui
xgb_ensemble_mcc = matthews_corrcoef(y_test, xgb_y_pred_xgb_ensemble_test_)
xgb_ensemble_kappa = cohen_kappa_score(y_test, xgb_y_pred_xgb_ensemble_test_)
xgb_ensemble_balanced_acc = balanced_accuracy_score(y_test, xgb_y_pred_xgb_ensemble_test_)

# ----------------------------------------------------------
# 2. MLP (Torch)
# ----------------------------------------------------------

# Normalizar os dados
X_train_scaled = scaler_mlp_torch.transform(X_train)
X_test_scaled = scaler_mlp_torch.transform(X_test)

# Converter para tensor do PyTorch
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# Criar a instância do modelo com os hiperparâmetros corretos
input_size = X_test.shape[1]
model_mlp_torch = MLP(input_size, hidden_layer_sizes, activation, dropout_rate)

# Carregar o modelo salvo
model_mlp_torch = torch.load(r"C:\Users\jgeov\OneDrive\Documentos\GitHub\Ciencia_de_dados-1\Churn_predict\MLP-Pytorch\best_model_inteiro.pth", map_location=device)
model_mlp_torch.to(device)
model_mlp_torch.eval()

# Fazer previsões para os dados de teste
with torch.no_grad():
    # Aplicar a função sigmoide para garantir que as probabilidades estão no intervalo [0, 1]
    y_pred_proba_mlp_ensemble_test = torch.sigmoid(model_mlp_torch(X_test_tensor)).cpu().numpy().flatten()

# Converter probabilidades para rótulos binários (0 ou 1) no conjunto de teste
y_pred_mlp_ensemble_test_ = (y_pred_proba_mlp_ensemble_test >= 0.5).astype(int)

# Fazer previsões para os dados de treino
with torch.no_grad():
    # Aplicar a função sigmoide para garantir que as probabilidades estão no intervalo [0, 1]
    y_pred_proba_mlp_ensemble_train = torch.sigmoid(model_mlp_torch(X_train_tensor)).cpu().numpy().flatten()

# Converter probabilidades para rótulos binários (0 ou 1) no conjunto de treino
y_pred_mlp_ensemble_train_ = (y_pred_proba_mlp_ensemble_train >= 0.5).astype(int)

# Agora, temos as probabilidades e as classes para treino e teste

# Calcular as métricas
mlp_ensemble_accuracy = accuracy_score(y_test, y_pred_mlp_ensemble_test_)
mlp_ensemble_precision = precision_score(y_test, y_pred_mlp_ensemble_test_)
mlp_ensemble_recall = recall_score(y_test, y_pred_mlp_ensemble_test_)
mlp_ensemble_f1 = f1_score(y_test, y_pred_mlp_ensemble_test_)
mlp_ensemble_auc_roc = roc_auc_score(y_test, y_pred_proba_mlp_ensemble_test)  # AUC-ROC agora usando as probabilidades (não as classes)
mlp_ensemble_mcc = matthews_corrcoef(y_test, y_pred_mlp_ensemble_test_)
mlp_ensemble_kappa = cohen_kappa_score(y_test, y_pred_mlp_ensemble_test_)
mlp_ensemble_balanced_acc = balanced_accuracy_score(y_test, y_pred_mlp_ensemble_test_)


# ----------------------------------------------------------
# 3. CatBoost
# ----------------------------------------------------------


# Previsões e métricas treino 
# Previsões no conjunto de treino
catboost_y_pred_proba_ensemble_train = model_catboost.predict_proba(X_train_scaled_catboost)[:, 1]
catboost_y_pred_ensemble_train_ = (catboost_y_pred_proba_ensemble_train >= 0.5).astype(int)

# Previsões e métricas teste 
catboost_y_pred_proba_ensemble_teste = model_catboost.predict_proba(X_test_scaled_catboost)[:, 1]
catboost_y_pred_ensemble_teste_ = (catboost_y_pred_proba_ensemble_teste >= 0.5).astype(int)

# Calcular as métricas
catboost_ensemble_accuracy = accuracy_score(y_test, catboost_y_pred_ensemble_teste_)
catboost_ensemble_precision = precision_score(y_test, catboost_y_pred_ensemble_teste_)
catboost_ensemble_recall = recall_score(y_test, catboost_y_pred_ensemble_teste_)
catboost_ensemble_f1 = f1_score(y_test, catboost_y_pred_ensemble_teste_)
catboost_ensemble_auc_roc = roc_auc_score(y_test, catboost_y_pred_proba_ensemble_teste)  # Correção aqui
catboost_ensemble_mcc = matthews_corrcoef(y_test, catboost_y_pred_ensemble_teste_)
catboost_ensemble_kappa = cohen_kappa_score(y_test, catboost_y_pred_ensemble_teste_)
catboost_ensemble_balanced_acc = balanced_accuracy_score(y_test, catboost_y_pred_ensemble_teste_)

# ----------------------------------------------------------
# Exibir as métricas para os três modelos
# ----------------------------------------------------------

import matplotlib.pyplot as plt
import pandas as pd

# Dados das métricas de cada modelo
data = {
    'Métrica': ['Acurácia', 'Precisão', 'Recall', 'F1-Score', 'AUC-ROC', 'MCC', 'Kappa de Cohen', 'Acurácia Balanceada'],
    'XGBoost': [xgb_ensemble_accuracy, xgb_ensemble_precision, xgb_ensemble_recall, xgb_ensemble_f1, xgb_ensemble_auc_roc, 
                xgb_ensemble_mcc, xgb_ensemble_kappa, xgb_ensemble_balanced_acc],
    'MLP (Torch)': [mlp_ensemble_accuracy, mlp_ensemble_precision, mlp_ensemble_recall, mlp_ensemble_f1, mlp_ensemble_auc_roc, 
                    mlp_ensemble_mcc, mlp_ensemble_kappa, mlp_ensemble_balanced_acc],
    'CatBoost': [catboost_ensemble_accuracy, catboost_ensemble_precision, catboost_ensemble_recall, catboost_ensemble_f1, catboost_ensemble_auc_roc, 
                 catboost_ensemble_mcc, catboost_ensemble_kappa, catboost_ensemble_balanced_acc]
}

# Criando o DataFrame
metrics_df = pd.DataFrame(data)

# Plotando a tabela com as métricas
fig, ax = plt.subplots(figsize=(10, 3))  # Aumentei a largura para caber melhor os dados
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=metrics_df.values, colLabels=metrics_df.columns, cellLoc='center', loc='center', colColours=["lightblue"]*4)

# Ajustando os parâmetros da tabela
table.auto_set_font_size(False)  # Não usar o tamanho automático da fonte
table.set_fontsize(10)  # Definindo o tamanho da fonte para ser menor e caber melhor
table.scale(1.2, 1.2)  # Aumentando o tamanho da tabela para ocupar mais espaço

# Ajustando o layout para remover qualquer espaçamento extra
plt.subplots_adjust(left=0.05, right=0.95, top=0.9, bottom=0.1)

plt.show()


# ----------------------------------------------------------
# Plotar as Curvas ROC AUC dos três modelos, incluindo treino
# ----------------------------------------------------------

plt.figure(figsize=(10, 7))

# ROC Curve - XGBoost (Treinamento)
fpr_xgb_train, tpr_xgb_train, _ = roc_curve(y_train, xgb_model.predict(xgb.DMatrix(X_train_scaled_xgb)))
plt.plot(fpr_xgb_train, tpr_xgb_train, label=f"XGBoost Treinamento (AUC = {roc_auc_score(y_train, xgb_model.predict(xgb.DMatrix(X_train_scaled_xgb))):.4f})", linestyle='--')

# ROC Curve - XGBoost (teste)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_prob_xgb_ensemble_test)
plt.plot(fpr_xgb, tpr_xgb, label=f"XGBoost Teste (AUC = {xgb_ensemble_auc_roc:.4f})")



# ROC Curve - MLP (PyTorch)  (Treinamento)
fpr_mlp_train, tpr_mlp_train, _ = roc_curve(y_train, model_mlp_torch(torch.tensor(X_train_scaled_mlp, dtype=torch.float32).to(device)).cpu().detach().numpy())
plt.plot(fpr_mlp_train, tpr_mlp_train, label=f"MLP (PyTorch) Treinamento (AUC = {roc_auc_score(y_train, model_mlp_torch(torch.tensor(X_train_scaled_mlp, dtype=torch.float32).to(device)).cpu().detach().numpy()):.4f})", linestyle='--')

# ROC Curve - MLP (PyTorch) (teste)
fpr_mlp, tpr_mlp, _ = roc_curve(y_test, y_pred_proba_mlp_ensemble_test)
plt.plot(fpr_mlp, tpr_mlp, label=f"MLP (PyTorch) Teste (AUC = {mlp_ensemble_auc_roc:.4f})")



# ROC Curve - CatBoost - (Treinamento)
fpr_catboost_train, tpr_catboost_train, _ = roc_curve(y_train, model_catboost.predict_proba(X_train_scaled_catboost)[:, 1])
plt.plot(fpr_catboost_train, tpr_catboost_train, label=f"CatBoost Treinamento (AUC = {roc_auc_score(y_train, model_catboost.predict_proba(X_train_scaled_catboost)[:, 1]):.4f})", linestyle='--')

# ROC Curve - CatBoost (teste)
fpr_catboost, tpr_catboost, _ = roc_curve(y_test, catboost_y_pred_proba_ensemble_teste)
plt.plot(fpr_catboost, tpr_catboost, label=f"CatBoost Teste (AUC = {catboost_ensemble_auc_roc:.4f})")



# Linha de Chance
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")  # Linha diagonal

plt.xlabel("Taxa de Falsos Positivos")
plt.ylabel("Taxa de Verdadeiros Positivos")
plt.title("Curvas ROC AUC (Teste e Treinamento)")
plt.legend(loc="lower right")
plt.show()

# ----------------------------------------------------------
# Plotar as Matrizes de Confusão dos três modelos em Subplots
# ----------------------------------------------------------

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Matriz de Confusão - XGBoost nos dados de teste
xgb_cm = confusion_matrix(y_test, xgb_y_pred_xgb_ensemble_test_)
sns.heatmap(xgb_cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0], 
            xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
axes[0].set_title("Matriz de Confusão - XGBoost  (TESTE)")
axes[0].set_xlabel('Previsões')
axes[0].set_ylabel('Valores Reais')

# Matriz de Confusão - MLP nos dados de teste
mlp_cm = confusion_matrix(y_test, y_pred_mlp_ensemble_test_)
sns.heatmap(mlp_cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[1], 
            xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
axes[1].set_title("Matriz de Confusão - MLP (PyTorch)  (TESTE)")
axes[1].set_xlabel('Previsões')
axes[1].set_ylabel('Valores Reais')

# Matriz de Confusão - CatBoost nos dados de teste 
catboost_cm = confusion_matrix(y_test, catboost_y_pred_ensemble_teste_)
sns.heatmap(catboost_cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[2], 
            xticklabels=['Classe 0', 'Classe 1'], yticklabels=['Classe 0', 'Classe 1'])
axes[2].set_title("Matriz de Confusão - CatBoost  (TESTE)")
axes[2].set_xlabel('Previsões')
axes[2].set_ylabel('Valores Reais')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
import seaborn as sns
from sklearn.linear_model import LogisticRegression


# ------------------------------------------------------
# Gerar previsões dos 3 modelos (predicoes separadas)
# ------------------------------------------------------

# Criar DataFrames de treino e teste para as previsões
df_predicoes_train = pd.DataFrame({
    'XGBoost_Prob': y_pred_prob_xgb_ensemble_train,
    'MLP_Torch_Prob': y_pred_proba_mlp_ensemble_train,
    'CatBoost_Prob': catboost_y_pred_proba_ensemble_train
}, index=X_train.index)  # Mantendo o mesmo índice do X_train

df_predicoes_test = pd.DataFrame({
    'XGBoost_Prob': y_pred_prob_xgb_ensemble_test,
    'MLP_Torch_Prob': y_pred_proba_mlp_ensemble_test,
    'CatBoost_Prob': catboost_y_pred_proba_ensemble_teste
}, index=X_test.index)  # Mantendo o mesmo índice do X_test

# Concatenar com os datasets originais
X_train_stacking = pd.concat([X_train, df_predicoes_train], axis=1)
X_test_stacking = pd.concat([X_test, df_predicoes_test], axis=1)

# Exibir as primeiras linhas para conferir se as probs estao vindo certo
print(X_train_stacking.head())
print(X_test_stacking.head())








# ------------------------------------------------------
# Gerar previsões dos 3 modelos (medias das probabilidades)
# ------------------------------------------------------

# Criar DataFrames de treino e teste para as previsões individuais
#df_predicoes_train = pd.DataFrame({
#    'XGBoost_Prob': y_pred_prob_xgb_ensemble_train,
#    'MLP_Torch_Prob': y_pred_proba_mlp_ensemble_train,
#    'CatBoost_Prob': catboost_y_pred_proba_ensemble_train
#}, index=X_train.index)  # Mantendo o mesmo índice do X_train

#df_predicoes_test = pd.DataFrame({
#    'XGBoost_Prob': y_pred_prob_xgb_ensemble_test,
#    'MLP_Torch_Prob': y_pred_proba_mlp_ensemble_test,
#    'CatBoost_Prob': catboost_y_pred_proba_ensemble_teste
#}, index=X_test.index)  # Mantendo o mesmo índice do X_test

#print(df_predicoes_train.head())
#print(df_predicoes_test.head())

# Calcular a média das probabilidades para treino e teste (mantendo apenas a média)
#df_predicoes_train_media = pd.DataFrame({
#    'Media_Prob': df_predicoes_train.mean(axis=1)
#}, index=X_train.index)  # Mantendo o mesmo índice do X_train

#df_predicoes_test_media = pd.DataFrame({
#    'Media_Prob': df_predicoes_test.mean(axis=1)
#}, index=X_test.index)  # Mantendo o mesmo índice do X_test

# Concatenar apenas as médias com os datasets originais
#X_train_stacking = pd.concat([X_train, df_predicoes_train_media], axis=1)
#X_test_stacking = pd.concat([X_test, df_predicoes_test_media], axis=1)


# Exibir as primeiras linhas para conferir se as probs estao vindo certo
#print('---------------------------------------------------------------------------' ) #so pra separar os prints
#print(X_train_stacking.head())
#print(X_test_stacking.head())










#testando modelos para serrem metamodels(aqui so anota mesmo)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

# Criar o modelo de regressão logística # BOM MAS NEM TANTO
#meta_model = LogisticRegression()

# Criar o modelo XGBoost # Razoavel 
#meta_model = XGBClassifier(random_state=42) 


# Criar o modelo LightGBM# bom mas insatisfatorio 
#meta_model = LGBMClassifier(random_state=42)





# AQUI PRECISA ACHAR UMA FORMA DE APLICAR O SCALE_POS_WEIGHT PARA O mlp kERAS 

In [ ]:
#verificando balanceametno final (morrer de certeza)
print(round(y_train.value_counts(normalize=True)*100,1))
#continuam naturalmente desbalanceadas 




#Calcular o peso ideal para scale_pos_weight quando esta desbalanceado: pode ser pelo calculo de 
# razão inversa das proporções das classes (boa pratica pra dar um norte de onde comecar a explorar o hiperparametro scale_pos_weight):

pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
pos_weight
 
# isso significa que a classe 0 (Negativa) é 3.86 vezes maior que a positiva (1)

# o estudo de scale_pos_weight melhor foi necessario por o modelo estava propenso a definir a maioria dos casos como negativo por conta dos baixos valores de probabilidade (muito pelo threshold tbm). 





# teste de Shapiro-Wilk nos dados de treinamento 
* Padronização vs Normalização:

    * Padronização (StandardScaler):

            Utilizada quando os dados possuem distribuições com outliers ou grandes variações.
            Transformação para média 0 e desvio padrão 1, preservando a distribuição dos dados.

    * Normalização (MinMaxScaler):

            Recomendável quando os dados têm uma faixa limitada de valores ou distribuição assimétrica.
            Útil para modelos com funções de ativação como sigmoide ou tanh que exigem entradas em um intervalo específico.
            Transforma os dados para um intervalo, geralmente [0, 1], garantindo que todas as variáveis fiquem na mesma escala.

    * Conclusão do este de SHAPIRO: 
    
            Dados não apresentam normalidade, a normalização (MinMaxScaler) foi a técnica escolhida. 


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro

# Definindo o número de colunas para os subplots
num_cols = 3  # O número de colunas de subplots
num_vars = len(X_train_stacking.select_dtypes(include=['float32','float64', 'int64']).columns)  # Número de variáveis
num_rows = (num_vars // num_cols) + (num_vars % num_cols > 0)  # Calculando o número de linhas de subplots

# Ajustando o tamanho dos gráficos (largura, altura) com base no número de subgráficos
fig_width = 5 * num_cols  # Largura proporcional ao número de colunas
fig_height = 5 * num_rows  # Altura proporcional ao número de linhas

# Criando os subplots com o tamanho ajustado
fig, axes = plt.subplots(num_rows, num_cols, figsize=(fig_width, fig_height))
axes = axes.flatten()  # Para garantir que podemos indexar de forma unificada

# Iterando pelas variáveis numéricas
for i, column in enumerate(X_train_stacking.select_dtypes(include=['float32','float64', 'int64']).columns):
    # Teste de Shapiro-Wilk
    stat, p = shapiro(X_train_stacking[column])

    # Plotando o histograma e KDE
    sns.histplot(X_train_stacking[column], kde=True, ax=axes[i], color='skyblue', stat='density')
    axes[i].set_title(f"{column} | p-value = {p:.4f}")
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Densidade')

    # Adicionando texto sobre o p-valor
    if p > 0.05:
        axes[i].text(0.05, 0.95, f"Normal: p > 0.05", transform=axes[i].transAxes, fontsize=12, color='green')
    else:
        axes[i].text(0.05, 0.95, f"Não normal: p < 0.05", transform=axes[i].transAxes, fontsize=12, color='red')

# Ajustando layout
plt.tight_layout(pad=0.5)  # Ajustando o espaçamento entre os subgráficos
plt.show()




In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
import numpy as np
import optuna
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
import matplotlib.pyplot as plt


# Verificar se há GPU disponível
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU disponível:", gpus)
else:
    print("GPU não disponível.")




# Definindo a barra de progresso
n_trials_ = 150
threshold= 0.5
progress_bar = tqdm(total=n_trials_, desc="Otimização em andamento", unit="trial")

# Definindo os pesos das métricas para otimização
weights_skf = {
    'Accuracy': 0.00,
    'f1': 0.10,
    'precision': 0.10,
    'recall': 0.60,
    'auc': 0.10,
    'balanced_acc': 0.10,
    'mcc': 0.00
}


scaler = MinMaxScaler() # inicia normalizador


smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42) # Inicia do BorderlineSMOTE



def objective(trial):
    global progress_bar
    global threshold 
    learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.01, log=True)
    n_hidden_units = trial.suggest_int('n_hidden_units', 128, 256)
    n_layers = trial.suggest_int('n_layers', 2, 4)
    dropout_rate = trial.suggest_float('dropout_rate', 0.4, 0.7)
    batch_size = trial.suggest_categorical('batch_size', [128, 256])
    epochs = trial.suggest_int('epochs', 30, 80)
    
    activation_function = trial.suggest_categorical('activation_function', ['relu', 'tanh'])
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'nadam'])

    weight_decay = trial.suggest_float('weight_decay', 0.0001, 0.1, log=True)
    l2_regularization = trial.suggest_float('l2_regularization', 0.001, 0.1, log=True)
    momentum = trial.suggest_float('momentum', 0.4, 0.95) if optimizer == 'sgd' else 0.0

    verbose = 0




    # Criando o modelo MLP
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=(X_train_stacking.shape[1],)))

    # Definindo o inicializador de pesos com base na função de ativação
    for _ in range(n_layers):
        if activation_function == 'relu':
            initializer = 'he_normal'
        elif activation_function == 'tanh':
            initializer = 'glorot_uniform'
        else:
            initializer = 'glorot_uniform'  # Opção padrão caso queira adicionar outras ativations

        model.add(layers.Dense(n_hidden_units, activation=activation_function, 
                                kernel_initializer=initializer,  # Usando o inicializador com base na função de ativação
                                kernel_regularizer=keras.regularizers.l2(l2_regularization)))
        model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(1, activation='sigmoid'))  # Camada de saída


    # Compilando o modelo
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    elif optimizer == 'nadam':
        opt = keras.optimizers.Nadam(learning_rate=learning_rate)  # com a taxa de aprendizado
    else:
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC', 'accuracy', 'Precision', 'Recall'])






    # Realizando a validação cruzada estratificada
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    global weights_skf
    weighted_scores = []
    auc_roc_list, accuracy_list, precision_list, recall_list, f1_list, mcc_list, kappa_list, balanced_acc_list = [], [], [], [], [], [], [], []

    for train_idx, val_idx in skf.split(X_train_stacking, y_train):
        X_train_fold, X_val_fold = X_train_stacking.iloc[train_idx], X_train_stacking.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Normalizando os dados de treino (sem vazamento de dados)
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_val_fold_scaled = scaler.transform(X_val_fold)  # Aplica a transformação nos dados de validação

        # Aplicando o BorderlineSMOTE para balancear a classe minoritária no treino
        X_train_fold_scaled, y_train_fold = smote.fit_resample(X_train_fold_scaled, y_train_fold)

        # Treinando o modelo
        model.fit(X_train_fold_scaled, y_train_fold, epochs=epochs, batch_size=batch_size, verbose=verbose)

        # Prevendo as probabilidades da classe 1
        y_pred_prob = model.predict(X_val_fold_scaled, verbose=verbose).flatten()
        # Garantir que não há NaN nas previsões
        y_pred_prob = np.nan_to_num(y_pred_prob)

        y_pred = (y_pred_prob >= threshold).astype(int)

        auc_roc_list.append(roc_auc_score(y_val_fold, y_pred_prob))
        accuracy_list.append(accuracy_score(y_val_fold, y_pred))
        precision_list.append(precision_score(y_val_fold, y_pred))
        recall_list.append(recall_score(y_val_fold, y_pred))
        f1_list.append(f1_score(y_val_fold, y_pred))
        mcc_list.append(matthews_corrcoef(y_val_fold, y_pred))
        kappa_list.append(cohen_kappa_score(y_val_fold, y_pred))
        balanced_acc_list.append(balanced_accuracy_score(y_val_fold, y_pred))

        # Métricas para otimizar
        metrics_mean = {
            "AUC-ROC": np.mean(auc_roc_list),
            "Accuracy": np.mean(accuracy_list),
            "Precision": np.mean(precision_list),
            "Recall": np.mean(recall_list),
            "F1-Score": np.mean(f1_list),
            "MCC": np.mean(mcc_list),
            "Kappa": np.mean(kappa_list),
            "Balanced Accuracy": np.mean(balanced_acc_list),
        }

        weighted_score = (
            weights_skf['f1'] * metrics_mean["F1-Score"] +
            weights_skf['precision'] * metrics_mean["Precision"] +
            weights_skf['recall'] * metrics_mean["Recall"] +
            weights_skf['auc'] * metrics_mean["AUC-ROC"] +
            weights_skf['balanced_acc'] * metrics_mean["Balanced Accuracy"] +
            weights_skf['Accuracy'] * metrics_mean["Accuracy"] +
            weights_skf['mcc'] * metrics_mean["MCC"]
        )
        weighted_scores.append(weighted_score)

    # Imprimir o resumo das métricas
    print("📊 MÉTRICAS DOS FOLDS")
    print("="*60)
    print(f"{'Métrica':<25} {'Valor final'}")
    print("-"*60)

    for metric, value in metrics_mean.items():
        print(f"{metric:<25} {value:.4f}")

    print("="*60)

    progress_bar.update(1)
    return np.mean(weighted_scores)

# Otimização com Optuna

sampler_ = optuna.samplers.TPESampler(n_startup_trials=10, 
                                      n_ei_candidates=50, 
                                      group=True,seed=42,
                                      multivariate=True)

study = optuna.create_study(direction='maximize', sampler=sampler_, pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=15))
study.optimize(objective, n_trials=n_trials_)  # Certifique-se de que n_trials_ está bem definido


best_params = study.best_params

# Melhores hiperparâmetros
print("📊 MELHORES HIPERPARÂMETROS ENCONTRADOS")
print("═" * 60)
for param, value in best_params.items():
    print(f"{param:<25}{value:<15}")
print("═" * 60)


### TREINAMETNO FINAL 

# Criando modelo final com os melhores parâmetros obtidos pelo Optuna 
final_model = keras.Sequential()

# Adicionando a camada de entrada
final_model.add(layers.InputLayer(input_shape=(X_train_stacking.shape[1],)))

# Inicializador de pesos para a camada final (sigmoid)
weight_initializer_final = 'glorot_uniform'  # Usando Glorot Uniform para a camada de saída sigmoid

# Inicializador de pesos para as camadas ocultas
if best_params['activation_function'] == 'relu':
    weight_initializer_hidden = 'he_normal'  # He Normal para ReLU
elif best_params['activation_function'] == 'tanh':
    weight_initializer_hidden = 'glorot_uniform'  # Xavier ou Glorot Uniform para Tanh

# Adicionando as camadas ocultas com os melhores parâmetros
for _ in range(best_params['n_layers']):
    final_model.add(layers.Dense(best_params['n_hidden_units'], 
                                 activation=best_params['activation_function'],
                                 kernel_initializer=weight_initializer_hidden,
                                 kernel_regularizer=keras.regularizers.l2(best_params['l2_regularization'] + best_params['weight_decay'])))  # Aplicando weight_decay aqui
    final_model.add(layers.Dropout(best_params['dropout_rate']))  # Dropout após cada camada densa

# Camada de saída
final_model.add(layers.Dense(1, activation='sigmoid', kernel_initializer=weight_initializer_final))  # Saída binária

# Escolhendo o otimizador com base nos melhores parâmetros
if best_params['optimizer'] == 'adam':
    opt = keras.optimizers.Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'sgd':
    opt = keras.optimizers.SGD(learning_rate=best_params['learning_rate'], momentum=best_params['momentum'])
else:
    opt = keras.optimizers.RMSprop(learning_rate=best_params['learning_rate'])

# Compilando o modelo
final_model.compile(optimizer=opt, 
                    loss='binary_crossentropy', 
                    metrics=['AUC', 'accuracy', 'Precision', 'Recall'])




###### TREINANDO MODELO FINAL


# Normalizando os dados de treinamento final
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_stacking)

# Aplicando o BorderlineSMOTE no treinamento final para balancear as classes
smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42)
X_train_scaled, y_train = smote.fit_resample(X_train_scaled, y_train)

# Criando o callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitorando a perda de validação
    patience=15,  # Se não houver melhoria por 15 épocas, o treinamento será interrompido
    restore_best_weights=True,  # Restaurando os melhores pesos do modelo
    verbose=0  # MENSAGENS 
)

# Treinando o modelo final com validação e capturando o histórico
history = final_model.fit(
    X_train_scaled, 
    y_train,
    epochs=best_params['epochs'],
    batch_size=best_params['batch_size'],
    verbose=0,  # Omitindo os logs a cada época
    validation_split=0.2,  # Usando 20% dos dados para validação
    callbacks=[early_stopping]  # Passando o callback de EarlyStopping
)

# Extraindo os valores da perda do histórico de treinamento
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(loss) + 1)

# Criando o gráfico da perda por época
plt.figure(figsize=(8, 5))
plt.plot(epochs_range, loss, label='Loss - Treinamento', color='blue')
plt.plot(epochs_range, val_loss, label='Loss - Validação', color='red', linestyle='dashed')

# Configurando os rótulos e título
plt.xlabel('Épocas')
plt.ylabel('Perda (Loss)')
plt.title('Evolução da Perda Durante o Treinamento')
plt.legend()
plt.grid(True)

# Exibindo o gráfico
plt.show()
########################################### ALTERADO ATE AQUI (TESTANTO)


# Normalizando os dados de teste
X_test_scaled = scaler.transform(X_test_stacking)

# Fazendo predições com o modelo final
y_pred_final_prob = final_model.predict(X_test_scaled,verbose=0).flatten()  # Obtendo as probabilidades

# Definindo o limiar para determinar a classe final
y_pred_final = (y_pred_final_prob >= threshold).astype(int)  # O limiar padrão é 0.5, mas pode ser ajustado



# Cálculo das métricas
accuracy = accuracy_score(y_test, y_pred_final)
precision = precision_score(y_test, y_pred_final)
recall = recall_score(y_test, y_pred_final)
f1 = f1_score(y_test, y_pred_final)
auc_roc = roc_auc_score(y_test, y_pred_final_prob)
mcc = matthews_corrcoef(y_test, y_pred_final)
kappa = cohen_kappa_score(y_test, y_pred_final)
balanced_acc = balanced_accuracy_score(y_test, y_pred_final)
gini = (2 * auc_roc - 1) * 100

# Exibir as métricas finais
print("📊 MÉTRICAS FINAIS - TESTE")
print("=" * 60)
print(f"{'Métrica':<25} {'Valor final'}")
print("-" * 60)
print(f"{'Acurácia':<25} {accuracy:.4f}")
print(f"{'Precisão':<25} {precision:.4f}")
print(f"{'Recall':<25} {recall:.4f}")
print(f"{'F1-Score':<25} {f1:.4f}")
print(f"{'AUC-ROC':<25} {auc_roc:.4f}")
print(f"{'MCC':<25} {mcc:.4f}")
print(f"{'Kappa':<25} {kappa:.4f}")
print(f"{'Acurácia Balanceada':<25} {balanced_acc:.4f}")
print(f"{'Gini':<25} {gini:.4f}")
print("=" * 60)

# Criando os gráficos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot da Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred_final)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negativo', 'Positivo'], yticklabels=['Negativo', 'Positivo'],
            cbar=False, ax=axes[0])
axes[0].set_title('Matriz de Confusão (TESTE) - MLP')
axes[0].set_xlabel('Previsão')
axes[0].set_ylabel('Real')

# Plot da curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_final_prob)
roc_auc = auc(fpr, tpr)
axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Taxa de Falsos Positivos')
axes[1].set_ylabel('Taxa de Verdadeiros Positivos')
axes[1].set_title('Curva ROC (TESTE) - MLP')
axes[1].legend(loc="lower right")

# Distribuição de probabilidades

# classe positiva
probs_pos = y_pred_final_prob  
# classe negativa
probs_neg = 1 - y_pred_final_prob

# Plot classe positiva 
sns.kdeplot(probs_pos, color='blue', ax=axes[2], label='Classe Positiva', fill=True, alpha=0.6)

# Plot classe negativa 
sns.kdeplot(probs_neg, color='red', ax=axes[2], label='Classe Negativa', fill=True, alpha=0.02, linewidth=0.30)

# Ajustando o gráfico
axes[2].set_title("Distribuição das Probabilidades para as Classes Positiva e Negativa (TESTE) - MLP Keras")
axes[2].set_xlabel("Probabilidade")
axes[2].set_ylabel("Densidade")
axes[2].legend()
# setando eixo de probabilidades entre 0 e 1
axes[2].set_xlim(0, 1)


# Exibindo o gráfico
plt.show()

# Ajustando o layout
plt.tight_layout()
plt.show()

#final_model.summary() #infos do modelo



#NAO CONSGUI CONFIGURAR GPU, MUITA IMCOMPATIBILIDADE DE BIBLIOTECAS



# Explicabilidade 

# SHAP Global

* SHAP Global é uma técnica de explicabilidade que ajuda a entender a importância de cada variável (feature) nas predições do modelo de maneira global, ou seja, avaliando o impacto médio de cada feature ao longo de todas as instâncias do conjunto de dados.




In [ ]:
import shap

# Criação do explicador SHAP

background_samples = shap.kmeans(X_train_scaled, 50)  # 100 clusters representativos

explainer = shap.KernelExplainer(lambda x: final_model.predict(x, verbose=0), background_samples,n_jobs=-1 )


# Calculando os valores SHAP para o conjunto de teste
shap_values = explainer.shap_values(X_test_scaled)

# Plotando o gráfico de importância global com o SHAP
shap.summary_plot(shap_values[0], X_test_scaled, feature_names=[f"Feature {i+1}" for i in range(X_test_scaled.shape[1])])

# LIME (Local) 

* LIME (Local Interpretable Model-agnostic Explanations) é uma técnica de explicabilidade que busca interpretar predições de modelos de forma local, ou seja, explicar como o modelo chegou a uma decisão específica para uma instância de dado individual. 

* Ao contrário de métodos globais, como o SHAP, que explicam a importância das features em todas as predições, o LIME foca na explicação de uma única predição.

# Criando o explicador LIME para um modelo LightGBM
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_test_stacking.values,  # Dados de treino
    training_labels=y_test.values,         # Rótulos de treino
    mode="classification",                  # Tipo de problema (classificação)
    class_names=["Negativo", "Positivo"],   # Nomes das classes
    feature_names=X_test_stacking.columns, # Nomes das features
    discretize_continuous=True              # Discretizar variáveis contínuas
)

# Selecionando uma instância para explicar
idx = 200  # Índice do exemplo/observação a ser explicado
instance = X_test_stacking.iloc[idx].values.reshape(1, -1)

# Função para obter probabilidades usando LightGBM (para classificação binária)
def predict_proba_fn(x):
    raw_preds = final_model.predict(x)  # Usando o método predict do LightGBM
    # Convertendo as margens (logits) para probabilidades
    probabilities = 1 / (1 + np.exp(-raw_preds))  # Sigmoide para classificação binária
    return np.array([1 - probabilities, probabilities]).T  # Retorna a probabilidade para cada classe

# Gerando explicação local com probabilidades de classe
explanation = explainer.explain_instance(
    instance.flatten(),  # Passando os valores da instância em um formato adequado
    predict_proba_fn,     # Usando a função que retorna as probabilidades
    num_features=40      # Número de características a serem mostradas
)

# Plotando a explicação
fig = explanation.as_pyplot_figure()
plt.show()

# Gerando gráfico de importância das características (se necessário)
# explanation.as_list()  # Para mostrar os valores, tipo print
